In [1]:
import os
os.environ['PYTHONHASHSEED']=str(0)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import tensorflow as tf
tf.compat.v1.set_random_seed(0)
import matplotlib.pyplot as plt
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
from keras.optimizers import adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.utils import to_categorical
import csv
MNIST_image_path = 'MNIST_data'


Using TensorFlow backend.


In [2]:
import optuna
from optuna.integration import KerasPruningCallback
tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.experimental.list_logical_devices('GPU')))

Num GPUs Available:  1


In [3]:
def load_MNIST_data(MNIST_image_path, filename):
    
    features = []
    labels = []
    
    with open(MNIST_image_path + '/' + filename + '.csv') as file:
        file_csv = csv.reader(file)
        
        next(file_csv, None)
        for row in file_csv:
        
            labels.append(int(row[0]))
            features.append(row[1:785])
            
    features = np.array(features)
    labels = np.array(labels)
    features = features.astype(np.float)
    print('Data size is {}'.format(features.shape))
    features = features.reshape(len(features), 28, 28)
    
    return features, labels

def load_MNIST_test_data(MNIST_image_path, filename):
    
    features = []
    labels = []
    
    with open(MNIST_image_path + '/' + filename + '.csv') as file:
        file_csv = csv.reader(file)
        
        next(file_csv, None)
        for row in file_csv:
        
            labels.append(int(row[0]))
            features.append(row[1:785])
            
    features = np.array(features)
    labels = np.array(labels)
    features = features.astype(np.float)
    print('Data size is {}'.format(features.shape))
    features = features.reshape(len(features), 28, 28)
    
    return features, labels
    

In [4]:
def create_model(trial):
    # dropout in each layer
    
    model = Sequential()   
    model.add(
        Conv2D(
            filters=trial.suggest_categorical("filters", [32, 64]),
            kernel_size= (3,3),
            padding = 'same',
            activation= 'relu',
            input_shape=(28,28,1),
            )
        )  
    n_cnn = trial.suggest_int("n_cnn", 1, 3)
    def conv(n_cnn):
        i = n_cnn
        if i > 0:
            num_filters1 = trial.suggest_categorical("n_filters_1".format(i), [32, 64])
            model.add(
                Conv2D(
                filters=num_filters1,
                kernel_size= (3,3),
                padding = 'same',
                activation= 'relu'
                )
            )
            if i > 1:
                num_filters2 = trial.suggest_categorical("n_filters_2".format(i), [64, 128])
                model.add(
                    Conv2D(
                    filters=num_filters2,
                    kernel_size= (3,3),
                    padding = 'same',
                    activation= 'relu'
                    )
                )
                if i > 2:
                    num_filters3 = trial.suggest_categorical("n_filters_3".format(i), [64, 128])
                    model.add(
                        Conv2D(
                        filters=num_filters3,
                        kernel_size= (3,3),
                        padding = 'same',
                        activation= 'relu'
                        )
                    )                 
    conv(n_cnn)
    model.add(MaxPooling2D(pool_size=(2,2)))   
    model.add(Flatten())
    n_layers = trial.suggest_int("n_layers", 1, 3)
    if n_layers > 0:
        num_hidden1 = trial.suggest_categorical("n_units_1", [128, 256, 512, 1024, 2048, 4096])
        model.add(Dense(num_hidden1, activation="relu"))
        dropout1 = trial.suggest_uniform("dropout_1", 0.1, 0.3)
        model.add(Dropout(rate=dropout1))
        if n_layers > 1:
            num_hidden2 = trial.suggest_categorical("n_units_2", [128, 256, 512, 1024, 2048, 4096])
            model.add(Dense(num_hidden2, activation="relu"))
            dropout2 = trial.suggest_uniform("dropout_2", 0.1, 0.3)
            model.add(Dropout(rate=dropout2))
            if n_layers > 2:
                num_hidden3 = trial.suggest_categorical("n_units_3", [128, 256, 512, 1024, 2048, 4096])
                model.add(Dense(num_hidden3, activation="relu"))
                dropout3 = trial.suggest_uniform("dropout_3", 0.1, 0.3)
                model.add(Dropout(rate=dropout3))  
    model.add(Dense(10, activation="softmax"))
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)       
    opt = SGD(lr = lr)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

In [5]:
def objective(trial):
    
    # clear clutter from previous session graphs.
    tf.keras.backend.clear_session()
    
    training_images, training_labels = load_MNIST_data(MNIST_image_path, 'train')
    training_images = training_images.reshape(training_images.shape[0], 28, 28, 1)
    training_images = training_images/255.0
    
    test_images, test_labels = load_MNIST_test_data(MNIST_image_path, 'test')
    test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
    test_images = test_images/255.0

    print('Not using data augmentation.')
    
    # Generate our trial model.
    model = create_model(trial)
    
    # Fit the model on the training data.
    # The KerasPruningCallback checks for pruning condition every epoch.
    history = model.fit(training_images, training_labels, 
              callbacks=[KerasPruningCallback(trial, "val_accuracy")],
              epochs = 10,
              validation_split = 0.1, verbose = 2, shuffle = False)
    

    # Evaluate the model accuracy on the test set.
    score = model.evaluate(test_images, test_labels, verbose=0)
    print ('Test loss: {}, Test accuracy: {}'.format(score[0], score[1]*100.0))
    return score[1]

In [6]:
sampler = optuna.samplers.TPESampler()
#sampler = optuna.integration.CmaEsSampler()
study = optuna.create_study(sampler = sampler, direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50)
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Data size is (60000, 784)
Data size is (10000, 784)
Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              12846080  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_______________________________________

[I 2020-05-06 17:31:13,853] Finished trial#0 resulted in value: 0.972100019454956. Current best value is 0.972100019454956 with parameters: {'filters': 32, 'n_cnn': 1, 'n_filters_1': 64, 'n_layers': 1, 'n_units_1': 1024, 'dropout_1': 0.2608015810271156, 'lr': 0.001984832130137853}.


Data size is (60000, 784)
Data size is (10000, 784)
Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              12847104  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_______________________________________

[I 2020-05-06 17:33:22,645] Finished trial#1 resulted in value: 0.978600025177002. Current best value is 0.978600025177002 with parameters: {'filters': 64, 'n_cnn': 1, 'n_filters_1': 32, 'n_layers': 2, 'n_units_1': 2048, 'dropout_1': 0.22284868028357846, 'n_units_2': 256, 'dropout_2': 0.24652841732701594, 'lr': 0.0070197387377269}.


Data size is (60000, 784)
Data size is (10000, 784)
Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1605888   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_______________________________________

...

Data size is (60000, 784)
Data size is (10000, 784)
Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              25694208  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_______________________________________

[I 2020-05-06 20:03:16,182] Finished trial#48 resulted in value: 0.9736999869346619. Current best value is 0.9890999794006348 with parameters: {'filters': 64, 'n_cnn': 2, 'n_filters_1': 32, 'n_filters_2': 64, 'n_layers': 3, 'n_units_1': 128, 'dropout_1': 0.2073469203477234, 'n_units_2': 256, 'dropout_2': 0.21580347022338162, 'n_units_3': 256, 'dropout_3': 0.22357712450138745, 'lr': 0.009773166944173461}.


Data size is (60000, 784)
Data size is (10000, 784)
Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 128)       36992     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_______________________________________

[I 2020-05-06 20:06:41,626] Finished trial#49 resulted in value: 0.9846000075340271. Current best value is 0.9890999794006348 with parameters: {'filters': 64, 'n_cnn': 2, 'n_filters_1': 32, 'n_filters_2': 64, 'n_layers': 3, 'n_units_1': 128, 'dropout_1': 0.2073469203477234, 'n_units_2': 256, 'dropout_2': 0.21580347022338162, 'n_units_3': 256, 'dropout_3': 0.22357712450138745, 'lr': 0.009773166944173461}.


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  0
  Number of complete trials:  50
Best trial:
  Value:  0.9890999794006348
  Params: 
    filters: 64
    n_cnn: 2
    n_filters_1: 32
    n_filters_2: 64
    n_layers: 3
    n_units_1: 128
    dropout_1: 0.2073469203477234
    n_units_2: 256
    dropout_2: 0.21580347022338162
    n_units_3: 256
    dropout_3: 0.22357712450138745
    lr: 0.009773166944173461


In [7]:
with open("Optuna_result_dict_TPE_MNIST.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Results"])
    for trial in study.trials:
        writer.writerow([trial.value])